# 理解程式碼內容與生成程式碼

以目前唯一支援 GPT-3 Completions API 的 gpt-35-turbo-instruct 模型示範

In [2]:
import os
from dotenv import load_dotenv
from openai import AzureOpenAI

# 載入環境變數
load_dotenv()

# 設定呼叫 Azure OpenAI Service API 所需連線資訊
azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
api_key=os.getenv("AZURE_OPENAI_API_KEY") 
api_version=os.getenv("AZURE_OPENAI_API_VERSION")

# 最簡之 API 呼叫
client = AzureOpenAI(
  azure_endpoint=azure_endpoint, 
  api_key=api_key,  
  api_version=api_version
)

# 透過環境變數取得所使用的模型部署名稱
model = os.getenv('COMPLETIONS_DEPLOYMENT_NAME')

# 呼叫自動完成 API
response  = client.completions.create(
  model= model,
  prompt="### Postgres SQL tables, with their properties:\n" \
         "#\n" \
         "# Employee(id, name, department_id) \n" \
         "# Department(id, name, address)\n" \
         "# Salary_Payments(id, employee_id, amount, date) \n" \
         "### " \
         "查詢列出過去 3 個月僱用超過 10 名員工的部門名稱 \n"
         "query: SELECT",
  temperature=0,
  max_tokens=150,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0,
  stop=["#",";"])

print(response.choices[0].text)

 Department.name
FROM Department
INNER JOIN Employee ON Department.id = Employee.department_id
INNER JOIN Salary_Payments ON Employee.id = Salary_Payments.employee_id
WHERE Salary_Payments.date >= CURRENT_DATE - INTERVAL '3 months'
GROUP BY Department.name
HAVING COUNT(Employee.id) > 10


嘗試使用 gpt-35-turbo 與 gpt-4 開始支援的 ChatCompletion API 來進行自動完成

In [3]:
model = os.getenv('CHAT_DEPLOYMENT_NAME')

response = client.chat.completions.create(
  model= model,
  messages = [ 
    {"role": "system", "content": "你是以正體中文回覆的機器人，不要使用 Markdown 語法回覆。"}, 
    {'role': 'user', 'content': "精簡的告訴我以下這個 SQL 是什麼意思??\n"
      'Code:' \
      "SELECT d.name FROM Department d JOIN Employee e ON d.id = e.department_id WHERE e.id IN (SELECT employee_id FROM Salary_Payments WHERE date > now() - interval '3 months') GROUP BY d.name HAVING COUNT(*) > 10\n" \
      'Answer:'    
    }
  ],
  temperature=0,
  max_tokens=250
)

print(response.choices[0].message.content) 

這個 SQL 查詢的意思是：

從 Department 表中選取部門名稱，這些部門必須符合以下條件：
1. 與 Employee 表進行連接，連接條件是 Department 表的 id 與 Employee 表的 department_id 相等。
2. Employee 表中的員工 id 必須在 Salary_Payments 表中，且這些員工在過去三個月內有薪資記錄。
3. 按部門名稱分組，並且只選取員工數量超過 10 的部門。
